In [1]:
import os
import openai
import tiktoken
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

os.environ["OPENAI_API_KEY"] = '개인 API KEY 입력'

# Tokenizer setup
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

# Load and process PDF
loader = PyPDFLoader("./TEST.pdf")
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function=tiktoken_len)
texts = text_splitter.split_documents(pages)

# Embeddings setup
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

# Vector store setup
docsearch = Chroma.from_documents(texts, hf)

# OpenAI model setup for retrieval QA
openai = ChatOpenAI(model_name="gpt-4",
                    streaming=True, callbacks=[StreamingStdOutCallbackHandler()],
                    temperature=0)

qa = RetrievalQA.from_chain_type(llm=openai,
                                 chain_type="stuff",
                                 retriever=docsearch.as_retriever(
                                    search_type="mmr",
                                    search_kwargs={'k': 3, 'fetch_k': 10}),
                                 return_source_documents=True)

# Prompt for generating questions and answers
class QAPair(BaseModel):
    question: str = Field(alias='QUESTION')
    answer: str = Field(alias='ANSWER')

prompt_template = PromptTemplate.from_template(
    """Context information is below. You are only aware of this context and nothing else.
    ---------------------

    {context}

    ---------------------
    Given this context, generate only questions based on the below query.
    You are an Teacher/Professor in {domain}. 
    Your task is to provide exactly **{num_questions}** question(s) for an upcoming quiz/examination. 
    You are not to provide more or less than this number of questions. 
    The question(s) should be diverse in nature across the document. 
    The purpose of question(s) is to test the understanding of the students on the context information provided.
    You must also provide the answer to each question. The answer should be based on the context information provided only.

    Restrict the question(s) to the context information provided only.
    QUESTION and ANSWER should be written in Korean. response in JSON format which contains the `question` and `answer`.
    ANSWER should be a complete sentence.

    #Format:
    ```json
    {{
    "QUESTION": "방사능 오염 선박평형수가 국내에 유입되지 않도록 어떤 조치가 시행되고 있습니까?",
    "ANSWER": "일본 동부 6개현에서 평형수를 주입 후 국내 입항 예정인 선박은 국내 입항 24시간 전까지 평형수 입항보고서를 제출해야 하며, 일본 동부 6개현에서 평형수를 주입 후 국내 배출 예정인 선박은 우리나라 관할수역 밖에서 평형수 교환 후 입항해야 합니다."
    }},
    {{
    "QUESTION": "지방해양수산청에서는 어떤 경우에 선박의 출항을 허가하나요?",
    "ANSWER": "지방해양수산청에서는 출항 전까지 제출된 자료를 검토하여 미배출 여부를 확인하고, 선박의 출항은 배출 여부 확인 후에 가능합니다."
    }},
    {{"QUESTION": "방사능 오염 선박평형수가 국내에 유입되지 않도록 어떤 조치가 시행되고 있습니까?", 
    "ANSWER": "일본 동부 6개현에서 평형수를 주입 후 국내 입항 예정인 선박은 국내 입항 24시간 전까지 평형수 입항보고서를 제출해야 하며, 일본 동부 6개현에서 평형수를 주입 후 국내 배출 예정인 선박은 우리나라 관할수역 밖에서 평형수 교환 후 입항해야 합니다."}}
    ```
    """
)

parser = JsonOutputParser(pydantic_object=QAPair)

# Combine prompt and parser into a chain
chain = (
    prompt_template
    | ChatOpenAI(
        model="gpt-4",
        temperature=0,
        streaming=True,
        callbacks=[StreamingStdOutCallbackHandler()],
    )
    | parser
)

qa_pair = []

for doc in texts:
    if doc.page_content:
        qa_pair.append(
            chain.invoke(
                {"context": doc.page_content, "domain": "포트미스 항만 포털 가이드북 내 자료", "num_questions": "3"}
            )
        )

print(qa_pair)


/Users/gimseong-eon/opt/anaconda3/envs/langchain/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/gimseong-eon/opt/anaconda3/envs/langchain/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the

```json
{
"QUESTION": "방사능 오염 선박평형수가 국내에 유입되지 않도록 어떤 조치가 시행되고 있습니까?",
"ANSWER": "일본 동부 6개현에서 평형수를 주입 후 국내 입항 예정인 선박은 국내 입항 24시간 전까지 평형수 입항보고서를 제출해야 하며, 일본 동부 6개현에서 평형수를 주입 후 국내 배출 예정인 선박은 우리나라 관할수역 밖에서 평형수 교환 후 입항해야 합니다."
},
{
"QUESTION": "평형수 입항보고서를 제출해야 하는 선박은 어떤 선박인가요?",
"ANSWER": "일본 동부 6개현에서 평형수를 주입 후 국내 입항 예정인 선박이 평형수 입항보고서를 제출해야 합니다."
},
{"QUESTION": "우리나라 관할수역 밖에서 평형수를 교환해야 하는 선박은 어떤 선박인가요?", 
"ANSWER": "일본 동부 6개현에서 평형수를 주입 후 국내 배출 예정인 선박은 우리나라 관할수역 밖에서 평형수를 교환해야 합니다."}
}
``````json
{
"QUESTION": "선박이 평형수를 배출하지 않았음을 입증할 수 있는 자료는 무엇인가요?",
"ANSWER": "선박평형수관리기록부, 항해일지, 선박평형수처리설비 운전기록, 평형수 탱크 용량(사진) 등의 자료를 제출하여 평형수를 배출하지 않았음을 입증할 수 있습니다."
},
{
"QUESTION": "선박이 평형수를 배출하지 않았음을 입증하는 자료는 어디에 제출해야 하나요?",
"ANSWER": "선박이 평형수를 배출하지 않았음을 입증하는 자료는 지방해양수산청 담당자 이메일 등을 통해 제출해야 합니다."
},
{
"QUESTION": "지방해양수산청에서는 어떤 절차를 통해 선박의 미배출 여부를 확인하나요?",
"ANSWER": "지방해양수산청에서는 출항 전까지 제출된 자료를 검토하여 선박의 미배출 여부를 확인합니다."
}
``````json
{
"QUESTION": "지방해양수산청에서는 어떤 절차를 통해 선박의 출항을 허가하나요?",
"ANSWER": "지방해양수산청에서는 출항 전까